In [24]:
import geopandas
import r5py
import shapely as shp
import pandas as pd
import geopandas as gpd
import numpy as np
import datetime

import os
import snman
from snman import osmnx_customized as oxc
from snman.constants import *

PERIMETER = '_accessibility_debug'

# Set these paths according to your own setup
data_directory = os.path.join('C:',os.sep,'Users','lballo','polybox','Research','SNMan','SNMan Shared','data_v2')
inputs_path = os.path.join(data_directory, 'inputs')
process_path = os.path.join(data_directory, 'process', PERIMETER)
outputs_path = os.path.join(data_directory, 'outputs', PERIMETER)

CRS_internal = 2056      # for Zurich

In [25]:
perimeters = snman.io.import_geofile_to_gdf(
    os.path.join(inputs_path, 'perimeters', 'perimeters.shp'),
    index='id'
)

perimeters.head()

,geometry
id,
hardbruecke,"POLYGON ((2681783.594 1250389.068, 2682538.818..."
meister_zurich,"POLYGON ((2692470.353 1257302.072, 2692744.903..."
waltikon,"POLYGON ((2688840.134 1243821.068, 2689177.359..."
zurich-birmensdorf,"POLYGON ((2679899.582 1247244.141, 2680185.844..."
zurich,"POLYGON ((2685056.896 1244517.810, 2684280.851..."


In [26]:
regions = perimeters[
    perimeters.index.isin([
        'zurich', 'ebc_zrh_v01_ex_zurich'
    ])
]

regions

,geometry
id,
zurich,"POLYGON ((2685056.896 1244517.810, 2684280.851..."
ebc_zrh_v01_ex_zurich,"MULTIPOLYGON (((2671482.755 1238555.241, 26714..."


In [34]:
accessibility = snman.io.import_geofile_to_gdf(
    os.path.join(outputs_path, 'accessibility_before_after.gpkg')
)

accessibility = gpd.sjoin(
    accessibility,
    regions,
    how='left',
    predicate='within'
).rename(columns={'index_right': 'region'})

accessibility

,accessibility_before,age,sex,maritalstatus,residencepermit,residentpermit,statent_id,accessibility_after,accessibility_after-before,geometry,region
0,4396.422581,11,2,1,-2,-2,68432474.0,4732.946092,336.523511,POINT (2684338.001 1247404.001),zurich
1,4017.607357,81,2,2,-2,-2,68782456.0,4058.349676,40.742319,POINT (2687768.001 1245636.001),zurich
2,3747.073127,5,1,1,-2,-2,68782456.0,3988.614178,241.541051,POINT (2687781.001 1245628.001),zurich
3,3919.844548,64,2,4,-2,-2,70242463.0,3919.365605,-0.478943,POINT (2702381.001 1246271.001),ebc_zrh_v01_ex_zurich
4,3922.642433,66,1,2,-2,-2,70242463.0,3922.207375,-0.435057,POINT (2702356.001 1246313.001),ebc_zrh_v01_ex_zurich
...,...,...,...,...,...,...,...,...,...,...,...
2760,3825.020666,57,2,2,-2,-2,67192406.0,3824.864699,-0.155967,POINT (2671854.001 1240591.001),ebc_zrh_v01_ex_zurich
2761,3822.387353,36,1,2,-2,-2,67762627.0,3815.375382,-7.011971,POINT (2677577.001 1262718.001),ebc_zrh_v01_ex_zurich
2762,3927.689252,59,1,2,-2,-2,67762627.0,3925.053412,-2.635839,POINT (2677584.001 1262653.001),ebc_zrh_v01_ex_zurich
2763,3903.901796,53,2,2,301,3,67762627.0,3900.449403,-3.452393,POINT (2677583.001 1262713.001),ebc_zrh_v01_ex_zurich


Add group labels

In [58]:
accessibility['x'] = 'all'

accessibility['age_group'] = accessibility.apply(
    lambda row: 'age<=25' if row['age'] <= 25 else 'age>=65' if row['age'] >= 65 else 'age:other',
    axis=1
)

accessibility['nationality'] = accessibility.apply(
    lambda row: 'nationality:swiss' if row['residencepermit'] <= -2 else 'nationality:other',
    axis=1
)

accessibility['maritalstatus_type'] = accessibility.apply(
    lambda row: 'maritalstatus:married' if row['maritalstatus'] == 2 else 'maritalstatus:other',
    axis=1
)

accessibility['sex_type'] = accessibility.apply(
    lambda row: 'sex:male' if row['sex'] == 1 else 'sex:female' if row['sex'] == 2 else 'sex:other',
    axis=1
)

fields = {
    'accessibility_before': ['count', 'mean', 'median', 'std'],
    'accessibility_after': ['count', 'mean', 'median', 'std'],
    'accessibility_after-before': ['count', 'mean', 'median', 'std']
}

accessibility_stats = {
    'region_and_age': accessibility.groupby(['region', 'age_group']).agg(fields),
    'region_and_nationality': accessibility.groupby(['region', 'nationality']).agg(fields),
    'region_and_sex': accessibility.groupby(['region', 'sex_type']).agg(fields),
    'all': accessibility.groupby('x').agg(fields)
}

res = pd.concat(accessibility_stats.values())
res = res.iloc[::-1]

In [59]:
res.to_excel(
    os.path.join(outputs_path, 'accessibility_results.xlsx'),
)

In [60]:
res

accessibility_before               \
                                                          count         mean   
all                                                        2765  4191.477948   
(zurich, sex:male)                                          431  4615.628935   
(zurich, sex:female)                                        401  4573.034656   
(ebc_zrh_v01_ex_zurich, sex:male)                          1004  4015.624256   
(ebc_zrh_v01_ex_zurich, sex:female)                         926  4021.853666   
(zurich, nationality:swiss)                                 568  4590.817929   
(zurich, nationality:other)                                 264  4604.312063   
(ebc_zrh_v01_ex_zurich, nationality:swiss)                 1383  3996.719858   
(ebc_zrh_v01_ex_zurich, nationality:other)                  547  4073.966517   
(zurich, age>=65)                                           108  4682.045077   
(zurich, age<=25)                                           183  4438.029391   
(zurich, age:other)                                         541  4630.873791   
(ebc_zrh_v01_ex_zurich, age>=65)                            337  4114.555474   
(ebc_zrh_v01_ex_zurich, age<=25)                            518  3884.534507   
(ebc_zrh_v01_ex_zurich, age:other)                         1075  4053.143422   

                                                                     \
                                                 median         std   
all                                         4217.217301  460.538414   
(zurich, sex:male)                          4642.937122  267.092234   
(zurich, sex:female)                        4588.202973  275.738714   
(ebc_zrh_v01_ex_zurich, sex:male)           4041.419801  407.821931   
(ebc_zrh_v01_ex_zurich, sex:female)         4019.161964  419.937807   
(zurich, nationality:swiss)                 4621.265598  270.849617   
(zurich, nationality:other)                 4623.346118  274.640466   
(ebc_zrh_v01_ex_zurich, nationality:swiss)  4002.215267  414.661340   
(ebc_zrh_v01_ex_zurich, nationality:other)  4096.441833  405.982082   
(zurich, age>=65)                           4707.036477  269.858941   
(zurich, age<=25)                           4438.532599  280.233628   
(zurich, age:other)                         4649.332530  248.539046   
(ebc_zrh_v01_ex_zurich, age>=65)            4167.292548  434.032560   
(ebc_zrh_v01_ex_zurich, age<=25)            3897.842619  382.529771   
(ebc_zrh_v01_ex_zurich, age:other)          4070.438907  406.319818   

                                           accessibility_after               \
                                                         count         mean   
all                                                       2765  4225.034923   
(zurich, sex:male)                                         431  4724.796904   
(zurich, sex:female)                                       401  4680.631611   
(ebc_zrh_v01_ex_zurich, sex:male)                         1004  4017.430184   
(ebc_zrh_v01_ex_zurich, sex:female)                        926  4022.690992   
(zurich, nationality:swiss)                                568  4696.051084   
(zurich, nationality:other)                                264  4719.559569   
(ebc_zrh_v01_ex_zurich, nationality:swiss)                1383  3997.607597   
(ebc_zrh_v01_ex_zurich, nationality:other)                 547  4076.454217   
(zurich, age>=65)                                          108  4713.757075   
(zurich, age<=25)                                          183  4606.641642   
(zurich, age:other)                                        541  4734.232083   
(ebc_zrh_v01_ex_zurich, age>=65)                           337  4115.308418   
(ebc_zrh_v01_ex_zurich, age<=25)                           518  3885.769040   
(ebc_zrh_v01_ex_zurich, age:other)                        1075  4054.720431   

                                                                     \
                                                 median    